In [52]:
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_core.messages import SystemMessage, HumanMessage
from sklearn.metrics.pairwise import cosine_similarity
from langchain.schema import AIMessage
import json
import re
from langchain_core.runnables import RunnableLambda
import base64
from PIL import Image
import io
import os
import pandas as pd
import numpy as np
from tabulate import tabulate
import ast

In [ ]:
llm = ChatOllama(model="gemma3")
directory = "photos_victor"

In [ ]:
def encode_image(image_path, max_size=(512, 512), quality=80):
    image = Image.open(image_path)

    # Redimensionner l'image
    image.thumbnail(max_size)

    # Convertir en bytes avec compression
    buffer = io.BytesIO()
    image.save(buffer, format="JPEG", quality=quality)

    # Encoder en Base64
    encoded_string = base64.b64encode(buffer.getvalue()).decode("utf-8")

    return encoded_string

In [ ]:
def extract_json(response_text):
    """
    Extrait la portion JSON (délimitée par {}) de la réponse textuelle pour seulement avoir le dictionnaire et non le texte généré par l'ia.
    """
    match = re.search(r'\{.*\}', response_text, re.DOTALL)
    if match:
        json_str = match.group()
        try:
            return json.loads(json_str)
        except Exception as e:
            print(f"Erreur lors du chargement du JSON : {e}")
            return None
    else:
        print("Aucun JSON trouvé dans la réponse.")
        return None

In [ ]:
def prompt_func(data):
    type_ = data["type"]
    text = data["text"]
    content_parts = []

    if type_ == "keywords":
        image = data["image"]
        image_part = {
            "type": "image_url",
            "image_url": f"data:image/jpeg;base64,{image}",
        }
        content_parts.append(image_part)

    #system_message = SystemMessage(content=data["system_message_text"])

    text_part = {"type": "text", "text": text}

    content_parts.append(text_part)

    human_message = HumanMessage(content=content_parts)

    #return [system_message, human_message]
    return [human_message]

In [ ]:
def call_func(chain, prompt):
    try:
        response = chain.invoke(prompt)

        if isinstance(response, AIMessage):
            response_text = response.content
        else:
            response_text = str(response)  # Conversion en string si nécessaire

        #print(f"Reponse du llm : {response_text}")

        return extract_json(response_text)

    except Exception as e:
        print(f"Erreur de parsing JSON : {e}. Nouvelle tentative...")
        return -1

In [ ]:
def get_image_paths(directory):
    allowed_extensions = {".jpg", ".jpeg", ".png"}
    image_paths = [os.path.join(directory, filename) for filename in os.listdir(directory) if os.path.splitext(filename)[1].lower() in allowed_extensions]
    return image_paths

In [ ]:
def create_df(image_paths):
    image_list = []
    for path in image_paths:
        image = Image.open(path)
        image_name = os.path.basename(path)
        exifdata = image._getexif()
        date_time, localisation = None, None
        if exifdata:
            for tag_id, value in exifdata.items():
                tag = Image.ExifTags.TAGS.get(tag_id, tag_id)
                if tag == "DateTime":
                    date_time = value
                elif tag == "GPSInfo":
                    localisation = value

            image_list.append((image_name, path, date_time, localisation))

        else:
            print("Aucune donnée EXIF trouvée.")

    df = pd.DataFrame(image_list, columns=["image_name", "path", "date_time", "localisation"])
    df["keywords"] = ""
    df["categories"] = ""

    return df

In [ ]:
def add_keywords_to_df(image_data, keywords_output):
    if keywords_output:
        # Mise à jour uniquement pour les images présentes dans keywords_output
        image_data["keywords"] = image_data.apply(
            lambda row: keywords_output[row["image_name"]]
            if row["image_name"] in keywords_output else row["keywords"], axis=1
        )
    else:
        print("Aucun mot clé fourni ! ")
    return image_data

In [ ]:
def add_categories_to_df(image_data, categories_output):
    if categories_output:
        #Inversion du dict : on associe une categorie a chaque image
        image_to_categories = {img: cat for cat, images in categories_output.items() for img in images}

        image_data.loc[image_data["image_name"].isin(image_to_categories.keys()), "categories"] = image_data["image_name"].map(image_to_categories)
    else:
        print("Aucune catégorisation trouvée !")

    return image_data

In [ ]:
def get_missing_values(dictionnary):
    missing_values = {}
    for key, value in dictionnary.items():
        if value is None or value == "" or value == "nan" or value == "None":
            missing_values.update({key: value})

    return missing_values

In [ ]:
def get_missing_path(paths, dictionnary):
    missing_paths = []
    for path in paths:
        image_name = os.path.split(path)[-1]
        #print(f"image_name : {image_name}")
        if image_name in dictionnary.keys() or path in dictionnary.keys():
            missing_paths.append(path)

    return missing_paths

In [ ]:
def get_missing_values_path(paths, dictionnary):
    missing_values = get_missing_values(dictionnary)
    missing_paths = get_missing_path(paths, missing_values)
    print(f"Images détectées avec valeurs manquantes : {missing_values.keys()}")
    print(f"Chemins renvoyés pour traitement : {missing_paths}")
    return missing_paths

In [ ]:
def checking_all_keywords(df):
    path_images_empty = []
    none_possibilities = [None, "", [], "None", ["None"]]
    for row in df.itertuples():
        keywords = row.keywords
        if isinstance(keywords, float) and pd.isna(keywords):
            path_images_empty.append(row.path)

        elif keywords in none_possibilities:
            path_images_empty.append(row.path)

    return path_images_empty

In [ ]:
def keywords_call(df, image_paths, keywords_chain):
    for i in range(0, len(image_paths)):
        print(f"Image {i} : {image_paths[i]}")
        image_b64 = encode_image(image_paths[i])
        image_name = os.path.basename(image_paths[i])
        #print("Image name donnée au model : ", image_name)

        wrong_json = True
        max_iter = 100
        while wrong_json and max_iter > 0:
            prompt = {
                "type": "keywords",
                "text": f"""Décris-moi l'image avec 5 mots-clés. Les mots-clés doivent en priorité inclure des actions, des objets et un lieu si identifiables. Les mots-clés doivent être en français et peuvent être des mots composés.
                Retourne le résultat au format JSON suivant: {{ "{image_name}" : ["mot-clé1", "mot-clé2", "mot-clé3", "mot-clé4", "mot-clé5"] }}""",
                "image": image_b64
            }

            keywords_output = call_func(keywords_chain, prompt)
            print(f"Keywords : {keywords_output}\n")

            if keywords_output is None:
                max_iter -= 1
                print(f"On re-essaie avec au maximum : {max_iter}\n")
            else:
                df = add_keywords_to_df(df, keywords_output)
                wrong_json = False

    return df

In [ ]:
def pipeline_keywords(image_paths):
    image_data = create_df(image_paths)
    new_image_paths = image_paths

    prompt_chain = RunnableLambda(prompt_func)
    keyword_chain = prompt_chain | llm

    all_keywords = False
    only_once = False

    while not all_keywords :
        print("Entree dans le while")
        image_data = keywords_call(image_data, new_image_paths, keyword_chain)

        if only_once:
            new_row = pd.DataFrame([{"image_name" : "IMG_20241228_132157.jpg","path": "photos_victor/IMG_20241228_132157.jpg"}])
            image_data = pd.concat([image_data, new_row], ignore_index=True)
            only_once = False

        new_image_paths = checking_all_keywords(image_data)
        print(f"Images à traiter après le passage : {new_image_paths}")

        if not new_image_paths:
            all_keywords = True

    return image_data

In [ ]:
image_paths = get_image_paths(directory)
image_data = pipeline_keywords(image_paths)

In [ ]:
print(tabulate(image_data, headers="keys", tablefmt="psql"))

In [ ]:
def checking_all_categories(df):
    keywords_empty_categories = {}
    none_possibilities = [None, "", [], "None", ["None"]]
    for row in df.itertuples():
        check = row.categories
        if isinstance(check, float) and pd.isna(check):
            keywords_empty_categories.update({row.image_name: row.keywords})
        elif check in none_possibilities:
            keywords_empty_categories.update({row.image_name: row.keywords})

    return keywords_empty_categories

In [ ]:
def get_cat_list(df, initial_cat_list):
    cat_list = initial_cat_list
    for row in df.itertuples():
        if row.categories not in cat_list:
            cat_list.append(row.categories)

    return cat_list

In [ ]:
def categories_call(df, keywords, limit_size, cat_list, cat_chain):
    for i in range(0, len(keywords), limit_size):
        interval = [i, min(i + limit_size, len(df))]

        subset_keys = list(keywords.keys())[interval[0]:interval[1]]

        subset_keywords = {key: keywords[key] for key in subset_keys}
        print(f"Keywords : {subset_keywords}")

        wrong_json = True
        max_iter = 100
        while wrong_json and max_iter > 0:
            prompt = {"type":"categories", "text":f"""
                1. Regroupe les images en fonction de l'action, évènement ou de l'activité qu'elles représentent.
                2. Chaque catégorie est définie par un seul mot-clé descriptif.
                3. Une image ne peut appartenir qu'à une seule catégorie.
                4. Priorise les catégories existantes : {cat_list}. Si une image correspond à l'une d'elles, classe-la dedans.
                5. Si aucune catégorie existante ne convient, crée une nouvelle catégorie proche d’une activité de voyage ou une catégorie plus générique (ex: "Nature", "Repas", "Loisirs")
                6. Réduis autant que possible la catégorie "Autres". N’y mets une image que si elle est vraiment impossible à classer ailleurs.

                - Listes de mots-clés détectés pour chaque image dans le format dict = "image_name": [mots-cles] : {subset_keywords}

                Retourne uniquement le résultat et au format JSON : {{ "categorie1": [ "image_name", "image_name" ], "categorie2": [ "image_name", "image_name" ],...}}"""
            }

            categories_output = call_func(cat_chain, prompt)
            print(f"Categories : {categories_output}\n")

            if categories_output is None:
                max_iter -= 1
                print(f"On re-essaie avec au maximum : {max_iter}\n")
            else:
                df = add_categories_to_df(df, categories_output)
                cat_list = get_cat_list(df, cat_list)
                print(f"Nouvelle liste des categories : {cat_list}")
                wrong_json = False

    return df

In [ ]:
def pipeline_categories(image_data, limit_size=20):
    new_keywords = image_data.set_index("image_name")["keywords"].to_dict()
    #print(new_keywords)
    cat_list = ["Paysage", "Ville", "Plage", "Randonnée", "Sport", "Musée", "Restaurant", "Autres"]

    prompt_chain = RunnableLambda(prompt_func)
    cat_chain = prompt_chain | llm

    all_categories = False
    only_once = False

    while not all_categories:
        image_data = categories_call(image_data, new_keywords, limit_size, cat_list, cat_chain)

        if only_once:
            image_data.loc[1, "categories"] = None
            only_once = False


        new_keywords = checking_all_categories(image_data)
        print(f"Kewords à repasser après checking : {new_keywords}")
        cat_list = get_cat_list(image_data, cat_list)

        if not new_keywords:
            all_categories = True

    return image_data

In [ ]:
copy_image_data = pipeline_categories(image_data, limit_size=22)

In [ ]:
print(tabulate(copy_image_data, headers="keys", tablefmt="psql"))

In [ ]:
csv_name = directory + ".csv"
copy_image_data.to_csv(csv_name, index=False)

## Tests Embeddings

In [74]:
# Catégories avec Embeddings
def categories_from_embeddings(df, embeddings_model="mxbai-embed-large", predefined_categories=None):
    embeddings = OllamaEmbeddings(model=embeddings_model)

    # Catégories prédéfinies par défaut
    if predefined_categories is None:
        predefined_categories = ["Paysage", "Ville", "Plage", "Randonnée", "Sport", "Musée", "Restaurant", "Autres"]

    df_keywords = df.dropna(subset=['keywords']).copy()

    # Convertir les mots-clés en texte
    def keywords_to_text(keywords):
        try:
            keywords_list = ast.literal_eval(keywords)
            return ' '.join(str(k).lower() for k in keywords_list if k)
        except Exception as e:
            print(f"Erreur lors de la conversion des keywords: {e}")
            return ""
        
    df_keywords['keyword_text'] = df_keywords['keywords'].apply(keywords_to_text)

    def assign_category(keywords):
        if not keywords or keywords.strip() == "":
            return "Autres"

        # Générer l'embedding
        try:
            keyword_embedding = embeddings.embed_documents([keywords])[0]
        except Exception as e:
            print(f"Erreur d'embedding: {e}")
            return "Autres"
            
        category_embeddings = embeddings.embed_documents(predefined_categories)
            
        # Calculer les similarités cosinus
        similarities = cosine_similarity([keyword_embedding], category_embeddings)[0]

        # Normalisation des similarités
        normalized_similarities = (similarities - np.min(similarities)) / (np.max(similarities) - np.min(similarities) + 1e-8)
    
        print(f"Keywords: {keywords}")
        for cat, sim in zip(predefined_categories, normalized_similarities):
            print(f"{cat}: {sim:.3f}")
        print("\n")
        
        # Mappings spécifiques pour affiner la classification
        general_mappings = {
            "Randonnée": [
                "forêt", "chemin", "sentier", "arbres", "montagne",
                "promenade", "parcours"],
            "Plage": [
                "sable", "mer", "océan", "côte", "littoral",
                "soleil", "baignade", "bord de mer"],
            "Ville": [
                "bâtiments", "architecture", "urbain", "métropole",
                "rue", "infrastructure", "quartier", "centre-ville"],
            "Restaurant": [
                "repas", "gastronomie", "cuisine", "plat", "menu",
                "dîner", "brunch", "alimentaire"],
            "Musée": [
                "art", "culture", "histoire", "exposition", "patrimoine",
                "galerie", "collection"],
            "Paysage": [
                "panorama", "vue", "décor", "nature", "étendue",
                "campagne"],
            "Sport": [
                "athlétisme", "compétition", "entraînement", "stade",
                "exercice", "match", "activité physique", "performance"]
        }

        # Vérification des mappings spécifiques
        keywords_lower = keywords.lower()
        for category, mapping_keywords in general_mappings.items():
            if any(mapping_kw in keywords_lower for mapping_kw in mapping_keywords):
                return category

        # Décision finale basée sur la similarité
        max_similarity = np.max(normalized_similarities)
        best_category_index = np.argmax(normalized_similarities)
        
        # Vérifier l'écart avec la catégorie "Autres"
        if "Autres" in predefined_categories:
            others_index = predefined_categories.index("Autres")
            others_similarity = normalized_similarities[others_index]
        else:
            others_similarity = 0

        difference = max_similarity - others_similarity
        
        # Si la différence est trop faible retourner "Autres"
        if difference < 0.2:
            return "Autres"
        else:
            return predefined_categories[best_category_index]
    
    df_keywords['categories'] = df_keywords['keyword_text'].apply(assign_category)
    
    categories_output = {}
    for category in set(df_keywords['categories']):
        categories_output[category] = df_keywords[df_keywords['categories'] == category]['image_name'].tolist()

    return df_keywords[['image_name', 'keywords', 'categories']]

In [75]:
data = {
    'image_name': ['img1.jpg', 'img2.jpg', 'img3.jpg', 'img4.jpg', 'img5.jpg', 'img6.jpg', 'img7.jpg'],
    'keywords': [
        "['Chemin', 'Forêt', 'Arbres', 'Sentier', 'Ombre']",
        "['conteneurs', 'lasagnes', 'plat', 'cuisine', 'cuisson']",
        "['Bâtiments', 'Gazon', 'Pavillon', 'Ciel', 'Espace']",
        "['Architecture', 'Voiture', 'Rue', 'Reflets', 'Pluie']",
        "['Chaussures', 'Clogs', 'Étagères', 'Couleurs', 'Magasin']",
        "['Pot', 'Beurre', 'Supermarché', 'Texture', 'Beurre']",
        "['Bâtiment', 'Panneau', 'Ville', 'Herbe', 'Éclairage']"
        ]
        }

df = pd.DataFrame(data)

result_df = categories_from_embeddings(df)  
result_df

Keywords: chemin forêt arbres sentier ombre
Paysage: 0.924
Ville: 0.841
Plage: 0.637
Randonnée: 1.000
Sport: 0.000
Musée: 0.512
Restaurant: 0.250
Autres: 0.952


Keywords: conteneurs lasagnes plat cuisine cuisson
Paysage: 0.588
Ville: 0.592
Plage: 0.930
Randonnée: 0.195
Sport: 0.000
Musée: 0.255
Restaurant: 1.000
Autres: 0.690


Keywords: bâtiments gazon pavillon ciel espace
Paysage: 0.185
Ville: 0.979
Plage: 0.819
Randonnée: 0.255
Sport: 0.000
Musée: 1.000
Restaurant: 0.261
Autres: 0.979


Keywords: architecture voiture rue reflets pluie
Paysage: 0.608
Ville: 0.958
Plage: 1.000
Randonnée: 0.638
Sport: 0.000
Musée: 0.801
Restaurant: 0.563
Autres: 0.529


Keywords: chaussures clogs étagères couleurs magasin
Paysage: 0.627
Ville: 0.541
Plage: 0.544
Randonnée: 0.386
Sport: 0.000
Musée: 0.848
Restaurant: 0.233
Autres: 1.000


Keywords: pot beurre supermarché texture beurre
Paysage: 1.000
Ville: 0.674
Plage: 0.764
Randonnée: 0.279
Sport: 0.000
Musée: 0.416
Restaurant: 0.545
Autres: 0.891




,image_name,keywords,categories
0,img1.jpg,"['Chemin', 'Forêt', 'Arbres', 'Sentier', 'Ombre']",Randonnée
1,img2.jpg,"['conteneurs', 'lasagnes', 'plat', 'cuisine', ...",Restaurant
2,img3.jpg,"['Bâtiments', 'Gazon', 'Pavillon', 'Ciel', 'Es...",Ville
3,img4.jpg,"['Architecture', 'Voiture', 'Rue', 'Reflets', ...",Ville
4,img5.jpg,"['Chaussures', 'Clogs', 'Étagères', 'Couleurs'...",Autres
5,img6.jpg,"['Pot', 'Beurre', 'Supermarché', 'Texture', 'B...",Autres
6,img7.jpg,"['Bâtiment', 'Panneau', 'Ville', 'Herbe', 'Écl...",Ville
